In [ ]:
function optmod = infotask_dyprog_AA_trials(trtype)

% infotask_dyprog_AA_trials
% used for greatbrainexp
% dynamic programming-style approach to calculate the optimal return for a set of cards 
% AA TRIALS ONLY
%
% note that the 'A' row always refers to the first card turned over,
% irrespective of location. This row contains cards 'I' and 'J'. The 'B'
% row is the other row, containing cards 'K' and 'L'.
%
% see end of function for contents of output arguments
%
% Laurence Hunt, August 2015

switch trtype
    case {0,2,4}
        chooserule = 'big';
    case {1,3,5}
        chooserule = 'small';
end
switch trtype
    case {0,1}
        rule = 'max';
    case {2,3}
        rule = 'sum';
    case {4,5}
        rule = 'product';
end

winpoints = 60;
losspoints = -50;
turncosts = [0 -10 -15 -20];

%% calculate row values, in card points

for i = 1:10
    for j = 1:10
        switch rule
            case 'sum'
                rowval(i,j) = i+j;
        end
    end
end

%% calculate whether row containing (i,j) would win for each combination, and assoicated points

winIJ = nan(10,10,10,10); %what you win if you choose the IJ row
winKL = nan(10,10,10,10); %what you win if you choose the KL row
for i = 1:10
    for j = 1:10
        for k = 1:10
            for l = 1:10
                switch chooserule
                    case 'big'
                        winIJ(i,j,k,l) = sign(rowval(i,j)-rowval(k,l));
                        winKL(i,j,k,l) = sign(rowval(k,l)-rowval(i,j));                        
                    case 'small'
                        winIJ(i,j,k,l) = sign(rowval(k,l)-rowval(i,j));
                        winKL(i,j,k,l) = sign(rowval(i,j)-rowval(k,l));
                end
            end
        end
    end
end
winIJ(winIJ==0) = nan; %this combination not possible - neither row wins
winKL(winKL==0) = nan; %this combination not possible - neither row wins
winIJ(winIJ==1) = winpoints;
winKL(winKL==1) = winpoints;
winIJ(winIJ==-1) = losspoints;
winKL(winKL==-1) = losspoints;

%% calculate total points for turning a certain number of cards, including costs

for i = 1:10
    for j = 1:10
        for k = 1:10
            for l = 1:10
                chooseIJvals(i,j,k,l,1) = winIJ(i,j,k,l) + sum(turncosts(1));   %choose IJ, turn 1 card
                chooseIJvals(i,j,k,l,2) = winIJ(i,j,k,l) + sum(turncosts(1:2)); %choose IJ, turn 2 cards
                chooseIJvals(i,j,k,l,3) = winIJ(i,j,k,l) + sum(turncosts(1:3)); %choose IJ, turn 3 cards
                chooseIJvals(i,j,k,l,4) = winIJ(i,j,k,l) + sum(turncosts(1:4)); %choose IJ, turn 4 cards
                
                chooseKLvals(i,j,k,l,1) = winKL(i,j,k,l) + sum(turncosts(1));   %choose KL, turn 1 card
                chooseKLvals(i,j,k,l,2) = winKL(i,j,k,l) + sum(turncosts(1:2)); %choose KL, turn 2 cards
                chooseKLvals(i,j,k,l,3) = winKL(i,j,k,l) + sum(turncosts(1:3)); %choose KL, turn 3 cards
                chooseKLvals(i,j,k,l,4) = winKL(i,j,k,l) + sum(turncosts(1:4)); %choose KL, turn 4 cards
            end
        end
    end
end

%% value of choosing at stage 4 (assuming optimal choice)
val4ch = max(chooseIJvals(:,:,:,:,4),...
    chooseKLvals(:,:,:,:,4));
val4state = val4ch;                                 %value of being in this state 4 - best action
opt4chIJ = sign(chooseIJvals(:,:,:,:,4)-chooseKLvals(:,:,:,:,4)); opt4chIJ(isnan(opt4chIJ)) = 0;
val4chIJ = chooseIJvals(:,:,:,:,4)-chooseKLvals(:,:,:,:,4); val4chIJ(isnan(val4chIJ)) = 0;
val4chIJfull = chooseIJvals(:,:,:,:,4);
val4chKLfull = chooseKLvals(:,:,:,:,4);


%% value of choosing at stage 3 (assuming optimal choice)
chooseIJmean = nanmean(chooseIJvals(:,:,:,:,3),4);  %value of choosing IJ at stage 3
chooseKLmean = nanmean(chooseKLvals(:,:,:,:,3),4);  %value of choosing KL at stage 3
val3ch = max(chooseIJmean,chooseKLmean);            %value of choosing best available option
val3turn = nanmean(val4state,4);                    %value of turning over last card
val3state = max(val3ch,val3turn);                   %value of being in this state 3 - best action
opt3chIJ = sign(chooseIJmean-chooseKLmean); opt3chIJ(isnan(opt3chIJ)) = 0;
val3chIJ = chooseIJmean-chooseKLmean; val3chIJ(isnan(val3chIJ)) = 0;
val3chIJfull = chooseIJmean;
val3chKLfull = chooseKLmean;

%% value of choosing at stage 2 (assuming optimal choice)
chooseIJmean = nanmean(reshape(chooseIJvals(:,:,:,:,2),...
    [10 10 100]),3);                                %value of choosing IJ at stage 2
chooseKLmean = nanmean(reshape(chooseKLvals(:,:,:,:,2),...
    [10 10 100]),3);                                %value of choosing KL at stage 2
val2ch = max(chooseIJmean,chooseKLmean);            %value of choosing best available option
val2turn = mean(val3state,3);                       %value of turning over last card - not quite right, as there is a choice of which card to turn over
val2state = max(val2ch,val2turn);                   %value of being in this state 2 - best action
opt2chIJ = sign(chooseIJmean-chooseKLmean); opt2chIJ(isnan(opt2chIJ)) = 0;
val2chIJ = chooseIJmean-chooseKLmean; val2chIJ(isnan(val2chIJ)) = 0;
val2chIJfull = chooseIJmean;
val2chKLfull = chooseKLmean;

%% value of choosing at stage 1 (assuming optimal choice)
chooseIJmean = nanmean(reshape(chooseIJvals(:,:,:,:,1),...
    [10 1000]),2);                                  %value of choosing IJ at stage 1
chooseKLmean = nanmean(reshape(chooseKLvals(:,:,:,:,1),...
    [10 1000]),2);                                  %value of choosing KL at stage 1
val1ch = max(chooseIJmean,chooseKLmean);            %value of choosing best available option
val1turn = nanmean(val2state,2);                        %value of turning over last card
val1state = max(val1ch,val1turn);                   %value of being in this state 1 - best action
opt1chIJ = sign(chooseIJmean-chooseKLmean); opt1chIJ(isnan(opt1chIJ)) = 0;
val1chIJ = chooseIJmean-chooseKLmean; val1chIJ(isnan(val1chIJ)) = 0;
val1chIJfull = chooseIJmean;
val1chKLfull = chooseKLmean;


%% %average expected value of each trial
EVtrial = mean(val1state);

%% optimal number of turns for each [i,j,k,l] combination

optturns = nan(size(val4ch));
for i = 1:10;
    for j = 1:10;
       for k = 1:10;
           for l = 1:10
               if val1ch(i)>val1turn(i)
                   optturns(i,j,k,l)=1;
                   if (val1chIJ(i)>0) %you should choose IJ if optimal
                       optoutcome(i,j,k,l) = chooseIJvals(i,j,k,l,1);
                   else
                       optoutcome(i,j,k,l) = chooseKLvals(i,j,k,l,1);
                   end
               elseif val2ch(i,j)>val2turn(i,j)
                   optturns(i,j,k,l)=2;
                   if (val2chIJ(i,j)>0) %you should choose IJ if optimal
                       optoutcome(i,j,k,l) = chooseIJvals(i,j,k,l,2);
                   else
                       optoutcome(i,j,k,l) = chooseKLvals(i,j,k,l,2);
                   end
                   optturns(i,j,k,l)=2;
               elseif val3ch(i,j,k)>val3turn(i,j,k)
                   optturns(i,j,k,l)=3;
                   if val3chIJ(i,j,k)>0
                       optoutcome(i,j,k,l) = chooseIJvals(i,j,k,l,3);
                   else
                       optoutcome(i,j,k,l) = chooseKLvals(i,j,k,l,3);
                   end
               else
                   if val4chIJ(i,j,k,l)>0
                       optoutcome(i,j,k,l) = chooseIJvals(i,j,k,l,4);
                   else
                       optoutcome(i,j,k,l) = chooseKLvals(i,j,k,l,4);
                   end
                   optturns(i,j,k,l)=4;
               end
           end
       end
    end
end


%%

val1ch_opt = val1ch;
val1turn_opt = val1turn;
val1state_opt = val1state;


%% assign output arguments

% the value of choosing (optimally), sampling, and the state values at stages 1-3:
optmod.val1ch = val1ch;
optmod.val1turn = val1turn;
optmod.val1state = val1state;
optmod.val2ch = val2ch;
optmod.val2turn = val2turn;
optmod.val2state = val2state;
optmod.val3ch = val3ch;
optmod.val3turn = val3turn;

%the best option to choose (opt), and the relative expected value of choosing the
%A row versus the B row (val), at each stage:
optmod.opt1chIJ = opt1chIJ;
optmod.val1chIJ = val1chIJ;
optmod.opt2chIJ = opt2chIJ;
optmod.val2chIJ = val2chIJ;
optmod.opt3chIJ = opt3chIJ;
optmod.val3chIJ = val3chIJ;
optmod.opt4chIJ = opt4chIJ;
optmod.val4chIJ = val4chIJ;

%the expected number of points from choosing the A row, at each stage:
optmod.val1chIJfull = val1chIJfull;
optmod.val2chIJfull = val2chIJfull;
optmod.val3chIJfull = val3chIJfull;
optmod.val4chIJfull = val4chIJfull;

%the expected number of points from choosing the B row, at each stage:
optmod.val1chKLfull = val1chKLfull;
optmod.val2chKLfull = val2chKLfull;
optmod.val3chKLfull = val3chKLfull;
optmod.val4chKLfull = val4chKLfull;

%the optimal number of turns, and optimal outcome:
optmod.optturns = optturns;
optmod.optoutcome = optoutcome;
end

In [ ]:
pkg load statistics;
output = infotask_dyprog_AA_trials(3);

In [ ]:
fieldnames(output)

ans =
{
  [1,1] = val1ch
  [2,1] = val1turn
  [3,1] = val1state
  [4,1] = val2ch
  [5,1] = val2turn
  [6,1] = val2state
  [7,1] = val3ch
  [8,1] = val3turn
  [9,1] = opt1chIJ
  [10,1] = val1chIJ
  [11,1] = opt2chIJ
  [12,1] = val2chIJ
  [13,1] = opt3chIJ
  [14,1] = val3chIJ
  [15,1] = opt4chIJ
  [16,1] = val4chIJ
  [17,1] = val1chIJfull
  [18,1] = val2chIJfull
  [19,1] = val3chIJfull
  [20,1] = val4chIJfull
  [21,1] = val1chKLfull
  [22,1] = val2chKLfull
  [23,1] = val3chKLfull
  [24,1] = val4chKLfull
  [25,1] = optturns
  [26,1] = optoutcome
}



These are the values of choosing best available option

In [ ]:
getfield(output,'val1ch')

ans =

   40.7937
   34.1729
   26.5628
   18.2309
    9.4595
    9.4595
   18.2309
   26.5628
   34.1729
   40.7937



These are the values of turning.

In [ ]:
getfield(output,'val1turn')

ans =

   34.752
   31.807
   29.097
   26.954
   25.886
   25.886
   26.954
   29.097
   31.807
   34.752



# Extracting optimal turns

In [ ]:
optturns = getfield(output,'optturns');
%save optturns.mat optturns -ascii;
save('optturns.mat','optturns')

In [ ]:
fid = fopen('ny.txt','wt');
fprintf(fid, 'Happy\nNew\nYear');
fclose(fid);

In [ ]:
sprintf('(%d,%d,%d,%d):%d\n',i,j,k,l,optturns(i,j,k,l));

In [ ]:
str1 = "Good";
str2 = "Morning";
strcat(str1, str2)

ans = GoodMorning


In [ ]:
opt_dict = containers.Map;
s = 10;
opt_string = "";
for i = 1:s
    for j = 1:s
        for k = 1:s
            for l = 1:s
                opt_string = strcat(opt_string, sprintf('(%d,%d,%d,%d):%d\n',i,j,k,l,optturns(i,j,k,l)));
            end
        end
    end
end

In [ ]:
fid = fopen('opt_string.txt','wt');
fprintf(fid, opt_string);
fclose(fid);

In [ ]:
save('optturns.mat','optdict')